# Count number of finguers using opencv 

<div class="alert alert-block alert-success">
How counting fingures works?
   <br> 
    
1. Grap ROI
      <br> 
2. Calculate average background value for 60 frames
      <br> 
3. Detect change in bacground
      <br> 
4. Once we detect the hand we will use Convex Hull algorthim 
      <br> 
5. Calculate center of the hand 
      <br> 
6. Calculate the angle of outer points to count fingure 
      <br> 
</div>



### Import Libraries 

In [1]:
import cv2
import numpy as np

from sklearn.metrics import pairwise

## Global Variables And Check Background 

- Define some global variables
- Creat function to calculate the average of the background values in ROI

In [2]:
# To updated when we calculate the avrage
backgorund = None

accumulated_weight = 0.5

# Calculating where the roi location
roi_top = 20
roi_bottom = 270
roi_right = 270
roi_left = 550

In [3]:
def calc_accum_avg(frame,accumulated_weight):
    
    global backgorund
    
    if backgorund is None:
        backgorund = frame.copy().astype('float')
        return None
    # The function calculates the weighted sum of the input image src
    # and the accumulator dst so that dst
    cv2.accumulateWeighted(frame,backgorund,accumulated_weight)

## Segmentation
` Useing thresholding to grab hand segment`


In [4]:
def segment(frame,threshold_min = 25):
    
    global backgorund
    
    # Calculate diffrance between bacground and the frame
    diff = cv2.absdiff(backgorund.astype('uint8'),frame)
    
    # Applying threshold on diff image
    _, thresholded = cv2.threshold(diff,threshold_min,255,cv2.THRESH_BINARY)
    
    # Find  external contours in thresholded image 
    image, contours, hierarchy = cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    # Check if we grab contours or not?
    if len(contours) == 0:
        return None
    
    # If we grab contour we will assume that the max external contour is the hand
    else:
        
        hand_segment = max(contours, key=cv2.contourArea)
        
        return (thresholded, hand_segment)

## Counting Finguers And ConvexHull Algorthim


Computing the convex hull means that a non-ambiguous and efficient representation of the required convex shape is constructed.
(https://en.wikipedia.org/wiki/Convex_hull_algorithms)

<img src="hand_convex.png">


<br>
- Calculate the extreme top, bottom, left, and right points 
<br>
- After that, calculate thier intersection and consider that as center of the hand
<br>
- Calculate the distance for the farest point from the center
<br>
- Then, using this distance and take 80-90% ratio to create circle

In [5]:
def count_fingers(thresholded, hand_segment):
    
    convex_hull = cv2.convexHull(hand_segment)
    
    # Grab the exterem points
    top    = tuple(convex_hull[convex_hull[:, :, 1].argmin()][0])
    bottom = tuple(convex_hull[convex_hull[:, :, 1].argmax()][0])
    left   = tuple(convex_hull[convex_hull[:, :, 0].argmin()][0])
    right  = tuple(convex_hull[convex_hull[:, :, 0].argmax()][0])
    
    # The center point of hand ( half way from  top and bottom And half way from left and right)
    cX = (left[0] + right[0]) // 2
    cY = (top[1] + bottom[1]) // 2
    
    # Calculate the distance between points and centers
    distance = pairwise.euclidean_distances([(cX, cY)], Y=[left, right, top, bottom])[0]
    
    # Grab the farest point from the center
    max_distance = distance.max()
    
    # Create a circle with 80% radius size of the max distance
    radius = int(0.9 * max_distance)
    circumference = (2*np.pi*radius)
    
     #  ROI of only that circle
    circular_roi = np.zeros(thresholded.shape[:2],dtype='uint8')
    
    cv2.circle(circular_roi,(cX, cY), radius, 255, 10)
    
    circular_roi = cv2.bitwise_and(thresholded,thresholded,mask=circular_roi)
    
    
    image, contours, hierarchy = cv2.findContours(circular_roi.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    
    count = 0
    
    for cnt in contours:
        
        # Grab each contour points and bounding rectangle for each of contour
        (x, y, w, h) = cv2.boundingRect(cnt)
        
        # If the contour is not very bottom or (is not wrist ) more than (center + center*0.25)
        out_of_wrist = ((cY + (cY * 0.25)) > (y + h))
        
        #  Number of points along the contour does not exceed 25% of the circumference of the circular ROI (otherwise we're counting points of
        limit_points = ((circumference * 0.25) > cnt.shape[0])
        
        if  out_of_wrist and limit_points:
            count += 1
            
    return count

## Read Vedio From CAM

In [6]:
cam = cv2.VideoCapture(0)

# Intialize a frame count
num_frames = 0

while True:
    
    # Read the frames
    ret, frame = cam.read()
    
    # Just for flip the frame
    frame = cv2.flip(frame, 1)
    
    # Making a copy to play with
    frame_copy = frame.copy()
    
    # Grap the region of interest we detemined it previously 
    roi = frame[roi_top:roi_bottom, roi_right:roi_left]
    
    # Change color of roi and Bluer the image
    gray = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(7,7),0)
    
    if num_frames < 60:
        # After 60 frame calculate background average 
        calc_accum_avg(gray,accumulated_weight)
        
        # before reach frame number 60 show this text on screen
        if num_frames <=59:
            cv2.putText(frame_copy, 'WAIT UNTIL CALCULATE BACKGROUND', (100,150),cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,0,255), 2)
            cv2.imshow("Finger Count",frame_copy)
    else:
        
        hand = segment(gray)
        
        if hand is not None:
            
            thresholded, hand_segment = hand
            
            # Draw contours around hand in real time
            cv2.drawContours(frame_copy, [hand_segment + (roi_right, roi_top)], -1, (255,0,0), 5)
            
            # Count the fingers
            fingers = count_fingers(thresholded, hand_segment)

            # Display count
            cv2.putText(frame_copy, str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

            # Also display the thresholded image
            cv2.imshow("Thesholded", thresholded)
     
    # Draw rectangle on the roi 
    cv2.rectangle(frame_copy, (roi_left, roi_top), (roi_right, roi_bottom), (0,0,255), 5)
    
    # Incerment number of frame
    num_frames +=1
    
    # Display the frame
    cv2.imshow('Fingure count', frame_copy)
    
    # Close windows with Esc
    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break

# Release the camera and destroy all the windows
cam.release()
cv2.destroyAllWindows()
    